In [11]:
%env WANDB_PROJECT=nlu_sentiment_analysis
!wandb login 2cad8a8279143c69ce071f54bf37c1f5a5f4e5ff
import wandb

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import itertools
import requests, re, string, datetime, copy
from functools import partial

import torch
import torchvision.transforms as T, torch.nn.functional as F, torch.nn as nn

from datasets import Dataset
from transformers import TrainingArguments, EarlyStoppingCallback, TrainerCallback
from transformers import Trainer
from transformers.modeling_outputs import SequenceClassifierOutput

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

PATH = "./data/Sentipolc16/"

env: WANDB_PROJECT=nlu_sentiment_analysis
wandb: Appending key for api.wandb.ai to your netrc file: /home/steve/.netrc


In [4]:
train = pd.read_csv(PATH + "training_set_sentipolc16.csv")
train.head()

,idtwitter,subj,opos,oneg,iro,lpos,lneg,top,text
0,122449983151669248,1,0,1,0,0,1,1,Intanto la partita per Via Nazionale si compli...
1,125485104863780865,1,0,1,0,0,1,1,"False illusioni, sgradevoli realtà Mario Monti..."
2,125513454315507712,1,0,1,0,0,1,1,"False illusioni, sgradevoli realtà #editoriale..."
3,125524238290522113,1,0,1,0,0,1,1,Mario Monti: Berlusconi risparmi all'Italia il...
4,125527933224886272,1,0,1,0,0,1,1,Mario Monti: Berlusconi risparmi all'Italia il...


In [5]:
file1 = open(PATH + "test_set_sentipolc16_gold2000.csv", 'r') 
Lines = file1.readlines()
 
test = []
for line in Lines:
  arr = line.split("\",")
  if len(arr) != 9:
    arr[8] = arr[8] + arr[9]  #to account for tweets containing the delimiter charachter that would create more splits than needed
    del arr[9:]
  for i in range(8):
    arr[i] = int(arr[i].strip("\""))
  test.append(arr)

test = pd.DataFrame(test, columns=train.columns)
test.head()

,idtwitter,subj,opos,oneg,iro,lpos,lneg,top,text
0,507074506880712705,0,0,0,0,0,0,2,"""Tra 5 minuti presentazione piano scuola del g..."
1,507075789456961536,1,1,0,0,1,0,2,"""\""@matteorenzi: Alle 10 appuntamento su http:..."
2,507077511902425088,1,0,1,0,0,1,2,"""#labuonascuola gli #evangelisti #digitali non..."
3,507079183315787777,0,0,0,0,0,0,2,"""Riforma scuola Tutto il discorso di Renzi su..."
4,507080190225563648,1,0,0,0,0,0,2,""".@matteorenzi @MiurSocial #labuonascuola bast..."


In [6]:
"""Tokenization classes for Italian AlBERTo models."""
import collections
import os


def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    with open(vocab_file, "r", encoding="utf-8") as reader:
        tokens = reader.readlines()
    for index, token in enumerate(tokens):
        token = token.rstrip("\n")
        vocab[token] = index
    return vocab

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens

    unpack_hashtags=True,  # perform word segmentation on hashtags

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons]
)

class AlBERTo_Preprocessing(object):
    def __init__(self, do_lower_case=True, **kwargs):
        self.do_lower_case = do_lower_case

    def preprocess(self, text):
        if self.do_lower_case:
            text = text.lower()
        text = str(" ".join(text_processor.pre_process_doc(text)))
        text = re.sub(r'[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
        text = re.sub(r'^\s', '', text)
        text = re.sub(r'\s$', '', text)
        return text

a = AlBERTo_Preprocessing(do_lower_case=True)
s = "#IlGOverno presenta le linee guida sulla scuola #labuonascuola - http://t.co/SYS1T9QmQN"
b = a.preprocess(s)
print(b)

/home/steve/anaconda3/envs/nlu/lib/python3.7/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading english - 1grams ...
Reading english - 2grams ...


/home/steve/anaconda3/envs/nlu/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...
<hashtag> il governo </hashtag> presenta le linee guida sulla scuola <hashtag> la buona scuola </hashtag> <url>


In [7]:
def k_steps_evidence(num_iterations, early_stopping_patience, training_args, net, training, testing, validating):
    np.random.seed(0)
    torch.manual_seed(0)

    keys = ["eval_loss", "eval_accuracy", "eval_f1", "eval_precision", "eval_recall"]
    metrics = { i: [] for i in keys}
    for i in range(num_iterations):        
        trainer = Trainer(
            model=net(3),
            args=training_args, 
            train_dataset=training.shuffle(seed=i), 
            eval_dataset=validating, 
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)] if early_stopping_patience is not None else None
        )
        trainer.train()
        res = trainer.evaluate(testing)
        for m in keys:
            metrics[m].append(res[m])

    for m in metrics.keys():
        print("{:18s}\t: {:.4} ± {:.4}".format(m, np.mean(metrics[m]), np.std(metrics[m])))

In [8]:
from transformers import AutoTokenizer, AutoModel

a = AlBERTo_Preprocessing(do_lower_case=True)
s: str = "#IlGOverno presenta le linee guida sulla scuola #labuonascuola - http://t.co/SYS1T9QmQN"
b = a.preprocess(s)

tok = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
pretrained_model = AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
tok.model_max_length = 128 #model.config.max_position_embeddings
tokens = tok.tokenize(b)
print(tokens)

['<', 'ha', '##shtag', '>', 'il', 'governo', '<', '/', 'ha', '##shtag', '>', 'presenta', 'le', 'linee', 'guida', 'sulla', 'scuola', '<', 'ha', '##shtag', '>', 'la', 'buona', 'scuola', '<', '/', 'ha', '##shtag', '>', '<', 'ur', '##l', '>']


In [9]:
def tokenize_function(examples):
    sa = tok(examples["text"], padding="max_length", truncation=True)
    return sa

def separate2united_labels(row):
    """
        Return a single scalar integer label associated to the polarity of the tweet.

        Negative -> 0
        Neutral  -> 1
        Positive -> 2
        Mixed    -> 3
    """
    if row["opos"] == 0 and row["oneg"] == 0:
        return 1
    elif row["oneg"] == 0 and row["opos"] == 1:
        return 2
    elif row["oneg"] == 1 and row["opos"] == 0:
        return 0
    else:
        return 3

#train set
dataset = pd.DataFrame({"text": train.text.apply(a.preprocess), "idx": train.index, "labels": train[["opos", "oneg"]].apply(separate2united_labels, axis=1)})
X_train, X_val = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset["labels"])

X_train = Dataset.from_pandas(X_train)
X_val = Dataset.from_pandas(X_val)

training = X_train\
                    .map(tokenize_function, batched=True)\
                    .filter(lambda example: example['labels'] != 3)\
                    .shuffle(seed=42)\
                    .with_format("torch")
validating = X_val\
                    .map(tokenize_function, batched=True)\
                    .filter(lambda example: example['labels'] != 3)\
                    .with_format("torch")


#test set
dataset = pd.DataFrame({"text": test.text.apply(a.preprocess), "idx": test.index, "labels": test[["opos", "oneg"]].apply(separate2united_labels, axis=1)})
dataset = Dataset.from_pandas(dataset)

testing = dataset\
                    .map(tokenize_function, batched=True)\
                    .filter(lambda example: example['labels'] != 3)\
                    .with_format("torch")

In [13]:
from torch.utils.data import DataLoader
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, f1_score

class EarlyStopping():    
    def __init__(self, min_delta = 0, patience = 0):        
        self.min_delta = min_delta
        self.patience = patience
        self.wait = 0
        self.stopped_epoch = 0
        self.best = -np.Inf
        self.stop_training = False
    
    def on_epoch_end(self, epoch, current_value):
        if np.greater((current_value - self.min_delta), self.best):
            self.best = current_value
            self.wait = 0
        else:
            self.wait += 1
            if self.wait > self.patience:
                self.stopped_epoch = epoch
                self.stop_training = True
        return self.stop_training


TRAIN_BATCH_SIZE = 64 
PREDICT_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64 
WEIGHT_DECAY = 0.01
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3
MAX_SEQ_LENGTH = 128
WARMUP_PROPORTION = 0.1
num_train_steps = int(len(training) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)+1
NUM_WARMUP_STEPS =  int(num_train_steps * WARMUP_PROPORTION)
RUN_NAME = "test_trainer"

## Torch loop

In [15]:
class MyNetMC(nn.Module):
    """
        Attach a FC layer on top of the BERT head in order to produce a classification output.

        The pooled_output output of BERT is basically a projection of the [CLS] embeddings via another FC layer (768 -> 768 hidden units).
        We stack another FC layer with Dropout on top of that, as reported in https://github.com/google-research/bert/blob/eedf5716ce1268e56f0a50264a88cafad334ac61/run_classifier.py#L574
    """
    def __init__(self, num_labels):
        super(MyNetMC2, self).__init__()

        self.num_labels = num_labels
        self.model = copy.deepcopy(pretrained_model)#AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
        self.dropout1 = nn.Dropout(0.1)
        self.linear1 = nn.Linear(768, 3)

        self.loss_fct = nn.CrossEntropyLoss()


    def forward(self, labels, input_ids, attention_mask, **args):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, **args)
        x = self.dropout1(outputs[1])
        logits = self.linear1(x)
        loss = self.loss_fct(logits, labels)
        return logits , loss
    
class MyNetMCTuned(nn.Module):
    """
        Attach a FC layer on top of the BERT head in order to produce a classification output.

        The pooled_output output of BERT is basically a projection of the [CLS] embeddings via another FC layer (768 -> 768 hidden units).
        We stack another FC layer without Dropout on top of that, as reported in https://github.com/google-research/bert/blob/eedf5716ce1268e56f0a50264a88cafad334ac61/run_classifier.py#L574
    """
    def __init__(self, num_labels):
        super(MyNetMC2, self).__init__()

        self.num_labels = num_labels
        self.model = copy.deepcopy(pretrained_model)
        self.linear1 = nn.Linear(768, 3)

        self.loss_fct = nn.CrossEntropyLoss()

    def forward(self, labels, input_ids, attention_mask, **args):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, **args)
        logits = self.linear(outputs[1])
        loss = self.loss_fct(logits, labels)
        return logits , loss

## Utility funcitons

In [16]:
from transformers import get_linear_schedule_with_warmup

device = "cuda:0"

def train_epoch(model, train_loader, optimizer, scheduler, epoch, logging):
    model.train()
    targets = []
    outputs = []
    cumulative_loss = 0.
    for i , data in tqdm(enumerate(train_loader, 0), total=len(train_loader)):        
        targets.extend(data["labels"].numpy())
        
        batch = {k: v.to(device) for k, v in data.items()}
        logits , loss = model(**batch)

        cumulative_loss += loss.detach()
        if (i+1) % 25 == 0 and logging:
            print(f'Epoch: {epoch}, Loss:  {cumulative_loss.item()/i}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        outputs.extend(logits.argmax(-1).cpu().detach().numpy().tolist())
    #print("Cum_loss_opos = ", model.cum_loss_opos.item() / len(train_loader))
    #print("Cum_loss_oneg = ", model.cum_loss_oneg.item() / len(train_loader))
    #model.reset_cum_loss()
    if logging: wandb.log({"train": {'loss': cumulative_loss.item() / len(outputs)}})
    return outputs, targets
    
        
def validation_epoch(model, epoch, val_loader, kind, logging):
    model.eval()
    targets = []
    outputs = []
    cumulative_loss = 0.
    with torch.no_grad():
        for _, data in enumerate(val_loader, 0):
            #ids            = data['input_ids'].to(device, dtype = torch.long)
            #mask           = data['attention_mask'].to(device, dtype = torch.long)
            #token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            #targets        = data['labels'].to(device, dtype = torch.long)
            
            batch = {k: v.to(device) for k, v in data.items()}
            logits , loss = model(**batch)
            cumulative_loss += loss.detach()
            if (i+1) % 25 == 0 and logging:
                print(f'Epoch: {epoch}, Loss:  {cumulative_loss.item()/i}')
            
            #outputs_opos , outputs_oneg , _ = model(input_ids=ids, attention_mask=mask, token_type_ids=token_type_ids, labels=targets)
            
            targets.extend(batch["labels"].cpu().detach().numpy())
            outputs.extend(logits.argmax(-1).cpu().detach().numpy().tolist())
    if logging: wandb.log({kind: {'loss': cumulative_loss.item() / len(outputs)}})
    return outputs, targets

## HPT

In [17]:
%%script echo skipping
import optuna
from optuna.trial import TrialState

def objective(trial):
    model = MyNetMCTuned(3).to(device)

    # Generate the optimizers.
    lr = trial.suggest_float("learning_rate", 2e-6, 2e-4, log=True)
    wd = trial.suggest_float("weight_decay", 1e-6, 1e-1)
    warmup = trial.suggest_float("warmup_steps", 0., 0.9, step=0.3),
    optimizer = torch.optim.AdamW(model.parameters(),
                              lr=lr,
                              weight_decay=wd, 
                              eps=1e-6)

    early_stopping = EarlyStopping(min_delta=0.005, patience=3)

    train_loader = DataLoader(training.with_format("torch", columns=["input_ids", "attention_mask", "labels", "token_type_ids"]), shuffle=True, batch_size=64)
    val_loader = DataLoader(validating.with_format("torch", columns=["input_ids", "attention_mask", "labels", "token_type_ids"]), batch_size=64)

    num_epochs = 15
    num_train_steps = int(len(train_loader) * num_epochs) + 1
    NUM_WARMUP_STEPS =  int(num_train_steps * warmup[0])
    scheduler = get_linear_schedule_with_warmup(optimizer, NUM_WARMUP_STEPS, num_train_steps)

    logging = False
    
    best_val = 0
    for epoch in range(num_epochs):
        outputs, targets = train_epoch(model, train_loader, optimizer, scheduler, epoch, logging)
        #f1_train = f1_score(targets, outputs, average='macro')
        
        outputs, targets = validation_epoch(model, epoch, val_loader, "val", logging)
        f1_val = f1_score(targets, outputs, average='macro')
        
        if f1_val > best_val:
            best_val = f1_val
        if early_stopping.on_epoch_end(epoch, f1_val):
            print("\n\nEARLY STOPPING AFTER {} EPOCHS".format(epoch))
            break

        #print(f"Val F1 = {f1_val}")
        trial.report(f1_val, epoch)
        
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    return best_val

In [18]:
best_hyper = None

In [ ]:
%%script echo skipping
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial
best_hyper = trial.params
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

## Training AlBERTo

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
model = MyNetMC(3).to(device)

if best_hyper is None:
    best_hyper = {'learning_rate': TRAIN_BATCH_SIZE, 'warmup_steps': WARMUP_PROPORTION, 'weight_decay': WEIGHT_DECAY}

optimizer = torch.optim.AdamW(model.parameters(),
                              lr=best_hyper["learning_rate"],
                              weight_decay=best_hyper["weight_decay"], 
                              eps=1e-6)

train_loader = DataLoader(training.with_format("torch", columns=["input_ids", "attention_mask", "labels", "token_type_ids"]), shuffle=True, batch_size=32)
val_loader = DataLoader(validating.with_format("torch", columns=["input_ids", "attention_mask", "labels", "token_type_ids"]), batch_size=64)
test_loader = DataLoader(testing.with_format("torch", columns=["input_ids", "attention_mask", "labels", "token_type_ids"]), batch_size=64)

num_epochs = NUM_EPOCHS
num_train_steps = int(len(train_loader) * num_epochs) + 1
NUM_WARMUP_STEPS =  int(num_train_steps * best_hyper["warmup_steps"])
scheduler = get_linear_schedule_with_warmup(optimizer, NUM_WARMUP_STEPS, num_train_steps)

logging = True
if logging:
    run = wandb.init(
      project='nlu_sentiment_analysis',
      name="AlBERTo-MC",
      entity='mcstewe',
      notes='',
      reinit=True,
      config=dict (
        epochs=num_epochs
      )
    )
    wandb.watch(model)

best_value = 0
for epoch in range(num_epochs):
    outputs, targets = train_epoch(model, train_loader, optimizer, scheduler, epoch, logging)
    f1_train = f1_score(targets, outputs, average='macro')
    outputs, targets = validation_epoch(model, epoch, val_loader, "val", logging=False)
    
    f1_val = f1_score(targets, outputs, average='macro')
    if f1_val > best_value:
        best_value = f1_val
        torch.save(model.state_dict(), ".data/models/alberto_multiclass.pt")    
    
    print(f"Val F1 = {f1_val}")
    outputs, targets = validation_epoch(model, epoch, test_loader, "test", logging=False)
    f1_test = f1_score(targets, outputs, average='macro')
    print("Test F1=", f1_test)
    if logging:
        wandb.log({"train": {'f1': f1_train, "lr": optimizer.param_groups[0]['lr']}})
        wandb.log({"val": {'f1': f1_val}})
        wandb.log({"test": {'f1': f1_test}})
    
if logging:
    run.finish()

## Training AlBERTo tuned

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
model = MyNetMCTuned(3).to(device)

if best_hyper is None:
    best_hyper = {'learning_rate': 3.000003529363845e-06, 'warmup_steps': 0.6, 'weight_decay': 0.000260393798851559}


# we will iterate through the layers of the network
#final_weights = []
#pretrained_weights = []
# for name, param in model.named_parameters():
#     if name.startswith('model'):
#         pretrained_weights.append(param)
#     else:
#         final_weights.append(param)
#optimizer = torch.optim.AdamW(params=[
#                                {'params': pretrained_weights, 'lr': 1e-5},
#                                {'params': final_weights, 'lr': 6e-5}],
#                              weight_decay=0.01, 
#                              eps=1e-6)
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=best_hyper["learning_rate"],
                              weight_decay=best_hyper["weight_decay"], 
                              eps=1e-6)
early_stopping = EarlyStopping(min_delta=0.005, patience=3)


train_loader = DataLoader(training.with_format("torch", columns=["input_ids", "attention_mask", "labels", "token_type_ids"]), shuffle=True, batch_size=32)
val_loader = DataLoader(validating.with_format("torch", columns=["input_ids", "attention_mask", "labels", "token_type_ids"]), batch_size=64)
test_loader = DataLoader(testing.with_format("torch", columns=["input_ids", "attention_mask", "labels", "token_type_ids"]), batch_size=64)

num_epochs = 15
num_train_steps = int(len(train_loader) * num_epochs) + 1
NUM_WARMUP_STEPS =  int(num_train_steps * best_hyper["warmup_steps"])
scheduler = get_linear_schedule_with_warmup(optimizer, NUM_WARMUP_STEPS, num_train_steps)

logging = True
if logging:
    run = wandb.init(
      project='nlu_sentiment_analysis',
      name="AlBERTo-MC-tuned",
      entity='mcstewe',
      notes='',
      reinit=True,
      config=dict (
        epochs=num_epochs
      )
    )
    wandb.watch(model)

best_value = 0
for epoch in range(num_epochs):
    outputs, targets = train_epoch(model, train_loader, optimizer, scheduler, epoch, logging)
    f1_train = f1_score(targets, outputs, average='macro')
    outputs, targets = validation_epoch(model, epoch, val_loader, "val", logging=False)
    
    f1_val = f1_score(targets, outputs, average='macro')
    if f1_val > best_value:
        best_value = f1_val
        torch.save(model.state_dict(), ".data/models/alberto_multiclass_tuned.pt")    
    if early_stopping.on_epoch_end(epoch, f1_val):
        print("\n\nEARLY STOPPING AFTER {} EPOCHS".format(epoch))
        break
    
    print(f"Val F1 = {f1_val}")
    outputs, targets = validation_epoch(model, epoch, test_loader, "test", logging=False)
    f1_test = f1_score(targets, outputs, average='macro')
    print("Test F1=", f1_test)
    if logging:
        wandb.log({"train": {'f1': f1_train, "lr": optimizer.param_groups[0]['lr']}})
        wandb.log({"val": {'f1': f1_val}})
        wandb.log({"test": {'f1': f1_test}})
    
if logging:
    run.finish()

# Evaluation

### AlBERTo

In [ ]:
#load best model's params
model = MyNetMC(3).to(device)
model.load_state_dict(torch.load(".data/models/alberto_multiclass.pt"))

In [ ]:
test_loader = DataLoader(testing.with_format("torch", columns=["input_ids", "attention_mask", "labels", "token_type_ids"]), batch_size=64)
preds , trues = validation_epoch(model, None, test_loader, "test", logging=False)

dataset_test = pd.DataFrame({"text": test.text.apply(a.preprocess), "idx": test.index, "labels": test[["opos", "oneg"]].apply(separate2united_labels, axis=1)})
dataset_test = dataset_test[dataset_test.labels != 3]

In [ ]:
for n , (i , row) in enumerate(dataset_test.iterrows()):
    if preds[n] != trues[n]:
        print(f"{row['text']} ------ true={row['labels']} pred={preds[n]}  \n")
        assert row['labels'] == trues[n]

In [ ]:
print(classification_report(trues, preds, target_names=["negative", "neutral", "positive"]))

cm = confusion_matrix(trues, preds, normalize=True)
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "neutral", "positive"]).plot(values_format="d")
plt.title("Test AlBERToMC - raw tweets")

### AlBERTo tuned

In [ ]:
#load best model's params
model = MyNetMCTuned(3).to(device)
model.load_state_dict(torch.load(".data/models/alberto_multiclass_tuned.pt"))

preds , trues = validation_epoch(model, None, test_loader, "test", logging=False)

In [ ]:
for n , (i , row) in enumerate(dataset_test.iterrows()):
    if preds[n] != trues[n]:
        print(f"{row['text']} ------ true={row['labels']} pred={preds[n]}  \n")
        assert row['labels'] == trues[n]

In [ ]:
print(classification_report(trues, preds, target_names=["negative", "neutral", "positive"]))

cm = confusion_matrix(trues, preds, normalize=True)
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["negative", "neutral", "positive"]).plot(values_format="d")
plt.title("Test AlBERToMC tuned - raw tweets")